# SuperAI Season 4 - Level 1 Hackathon - Image Search

## CLIP VisionModel For Encode Image 

In [10]:
from PIL import Image
import requests
from transformers import AutoProcessor, CLIPModel

In [11]:
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = AutoProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [12]:
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

In [17]:
inputs = processor(images = image, return_tensors = "pt")
image_features = model.get_image_features(**inputs)

In [15]:
image_features.shape

torch.Size([1, 512])

## Solution-CLIP Zeroshot For Image Similarity Search 

### Cosine Similarity

In [1]:
import numpy as np

def cosine_similarity (A , B) :

    maximum_similarity = 0
    target_class = -1

    norm_A = np.linalg.norm(A)

    for i in range(B.shape[0]) :

        norm_B = np.linalg.norm(B[i])
        similarity_score = (np.dot(A , B[i].T) / (norm_A * norm_B))[0]


        print(similarity_score)

        if similarity_score > maximum_similarity :
            
            maximum_similarity = similarity_score
            target_class = i 

    return maximum_similarity , target_class

### Define Model CLIP

In [2]:
from PIL import Image
import requests
from transformers import AutoProcessor , CLIPModel

In [3]:
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = AutoProcessor.from_pretrained("openai/clip-vit-base-patch32")

### Read queries folder image and store to numpy array 

In [4]:
from PIL import Image
import os
import numpy as np

In [5]:
queries_path = 'datasets/queries/queries'
images = []

for file_name in os.listdir(queries_path) :

    image = Image.open(f'{queries_path}/{file_name}')
    images.append(image)

inputs = processor(images = images , return_tensors = 'pt')
image_features = model.get_image_features(** inputs)

In [6]:
image_features = image_features.detach().numpy()

In [7]:
with open('database/queries_matrix.npy' , 'wb') as file :

    np.save(file , image_features)
    

### Read test folder images to inference

In [8]:
from tqdm import tqdm
import pandas as pd

In [9]:
with open('database/queries_matrix.npy' , 'rb') as file :

    queries_matrix = np.load(file)

In [10]:
queries_matrix.shape

(22, 512)

In [11]:
submissions = {
    'img_file' : [] ,
    'class' : []
}

In [12]:
test_path = 'datasets/test/images'

for file_name in tqdm(os.listdir('datasets/test/images')) : 
    
    print(file_name)
    
    image = Image.open(f'{test_path}/{file_name}')
    inputs = processor(images = image , return_tensors = 'pt')
    image_features = model.get_image_features(** inputs)
    
    cosine_score , target_class = cosine_similarity(queries_matrix , image_features.detach().numpy())

    print(cosine_score , target_class)

  0%|          | 0/1120 [00:00<?, ?it/s]

001fbc27-2f26-4a68-8ce1-892586598b6f.jpg


: 

## Solution-CLIP Fewshot For Image Similarity Search